# Data Processing

1. Read .csv files
2. Clean tables
3. Merge Tables
4. Feature Selection

In [1]:
import pandas as pd

Read all files and rename columns to ...

In [2]:
#Reading CSVs and renaming repeated columns

account = pd.read_csv("data/account.csv", sep=";")
account.rename(columns={"date":"date_acc"}, inplace=True)

card_test = pd.read_csv("data/card_test.csv", sep=";")
#rename card type
card_train = pd.read_csv("data/card_train.csv", sep=";")

client = pd.read_csv("data/client.csv", sep=";")

disposition = pd.read_csv("data/disp.csv", sep=";")
disposition.rename(columns={"type":"type_disp"}, inplace=True)

district = pd.read_csv("data/district.csv", sep=";")
district.rename(columns={"code ": "district_id"}, inplace=True) #So we can merge later

loan_test = pd.read_csv("data/loan_test.csv", sep=";")
loan_test.rename(columns={"date":"date_loan", "amount":"amount_loan"}, inplace=True)
loan_train = pd.read_csv("data/loan_train.csv", sep=";")
loan_train.rename(columns={"date":"date_loan", "amount":"amount_loan"}, inplace=True)

trans_test = pd.read_csv("data/trans_test.csv", sep=";")
trans_test.rename(columns={"date":"date_trans", "type":"type_trans", "amount":"amount_trans"}, inplace=True)
trans_train = pd.read_csv("data/trans_train.csv", sep=";")
trans_train.rename(columns={"date":"date_trans", "type":"type_trans", "amount":"amount_trans"}, inplace=True)

/home/dani/miniconda3/envs/ac/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Merge tables

In [3]:
# account.merge(loan_train, on='account_id').merge(disposition, on='account_id').merge(card_train, on='disp_id')
train_data = loan_train
# train_data = train_data.drop(columns=['account_id','frequency', 'type_disp', 'type'])

test_data = loan_test # account.merge(loan_test, on='account_id').merge(disposition, on='account_id').merge(card_test, on='disp_id')
# test_data = test_data.drop(columns=['account_id', 'frequency', 'type_disp', 'type'])



In [4]:
test_data

,loan_id,account_id,date_loan,amount_loan,duration,payments,status
0,5895,4473,970103,93960,60,1566,NaN
1,7122,10365,970104,260640,36,7240,NaN
2,6173,5724,970108,232560,48,4845,NaN
3,6142,5591,970121,221880,60,3698,NaN
4,5358,2018,970121,38520,12,3210,NaN
...,...,...,...,...,...,...,...
349,4989,105,981205,352704,48,7348,NaN
350,5221,1284,981205,52512,12,4376,NaN
351,6402,6922,981206,139488,24,5812,NaN
352,5346,1928,981206,55632,24,2318,NaN


**ToDo** : Pôr os ficheiros de baixo dentro de uma pasta `modal_data`

In [5]:
train_data.to_pickle("train_data.pkl")
test_data.to_pickle("test_data.pkl")